In [1]:
import platform
import os

system = platform.system().lower().strip()
if system in ['darwin', 'windows']:
    path_root = '.'
elif system == 'linux':
    from google.colab import drive
    drive.mount('/content/drive')

    path_root = './drive/MyDrive/code/03.crawler/00.Tools'
else:
    print('fuck')

path_root

'.'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

import shutil
directory_out = os.path.join(path_root, './download')
if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)
import re

# = = =  = = =  = = =

from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

# = = =  = = =  = = =

import json

# = = =  = = =  = = =

import requests

import sys
sys.path.append('./')
from crawler_configuration import get_header, get_proxy

# = = =  = = =  = = =

from PIL import Image
from io import BytesIO

# = = =  = = =  = = =

directory_in = os.path.join(path_root, './crawler')
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['0.null.txt', '.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
for file in list_file:

    directory_file = os.path.join(directory_out, file.removesuffix('.xlsx'))
    os.makedirs(directory_file)

    # = = =  = = =  = = =

    print('Data Loading...')
    wb = load_workbook(os.path.join(directory_in, file))
    ws = wb.worksheets[0]
    print('Loading Done !')
    print()

    for column in range(1, ws.max_column + 1):
        if ws.cell(1, column).value == 'Json_Src':
            column_src = column

    crawler_remain = ws.max_row - 1

    print(f'总数量：{crawler_remain}')
    print()

    # = = =  = = =  = = =

    def crawler(crawler_index):
        global crawler_remain

        crawler_status = False

        # = = =  = = =  = = =

        try:
            url_request = '' if not ws.cell(crawler_index, column_src).value or ws.cell(crawler_index, column_src).value == '' else json.loads(ws.cell(crawler_index, column_src).value).get('0', '')

            # = = =  = = =  = = =

            if url_request == '':
                crawler_retry = 0
            else:
                crawler_retry = 0
                while True:
                    crawler_retry += 1

                    try:
                        resp = requests.get(url_request,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(5, 15))

                        if resp.status_code == 200:
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = =  = = =  = = =

                image = Image.open(BytesIO(resp.content)).convert('RGB')

                # = = =  = = =  = = =

                buffer = BytesIO()
                image.save(buffer, format="PNG")
                buffer.seek(0)

                with open(os.path.join(directory_file, f'./{crawler_index - 1}.png'), 'wb') as pic:
                    pic.write(buffer.getvalue())

            # = = =  = = =  = = =

            crawler_status = True
        except:
            pass

        # = = =  = = =  = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and crawler_remain == 1 or crawler_remain % 30 == 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{crawler_retry}次] - [剩余{crawler_remain - 1}条] - [{file.removesuffix('.xlsx')} ({(list_file.index(file) + 1) / len(list_file) * 100:.2f}%)] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {ws.cell(crawler_index, 1).value}. {url_request}\n''')

        with semaphore_remain:
            crawler_remain -= 1

    # = = =  = = =  = = =

    pool.map(crawler, range(2, ws.max_row + 1))

# = = =  = = =  = = =

print('Done ~')

[True] - [请求1次] - [剩余29条] - [3.part (100.00%)] - [2024/12/02 09:36:46] - 68. https://m.media-amazon.com/images/I/611iga8GoDL._AC_SL1200_.jpg

[True] - [请求1次] - [剩余28条] - [3.part (100.00%)] - [2024/12/02 09:36:46] - 61. https://m.media-amazon.com/images/I/51p+h5VGLtL._SL1500_.jpg

[True] - [请求1次] - [剩余27条] - [3.part (100.00%)] - [2024/12/02 09:36:46] - 63. https://m.media-amazon.com/images/I/61xiZDJ7oAS._SL1500_.jpg

[True] - [请求1次] - [剩余26条] - [3.part (100.00%)] - [2024/12/02 09:36:46] - 64. https://m.media-amazon.com/images/I/61zfqfkGIpL._SL1500_.jpg

[True] - [请求1次] - [剩余25条] - [3.part (100.00%)] - [2024/12/02 09:36:47] - 65. https://m.media-amazon.com/images/I/71cULX4cTyL._SL1500_.jpg

[True] - [请求1次] - [剩余24条] - [3.part (100.00%)] - [2024/12/02 09:36:47] - 93. https://m.media-amazon.com/images/I/714Mvm1FfAL._AC_SL1500_.jpg

[True] - [请求1次] - [剩余23条] - [3.part (100.00%)] - [2024/12/02 09:36:48] - 72. https://m.media-amazon.com/images/I/617H3q0EWSL._SL1500_.jpg

[True] - [请求1次] - [剩余